# The Ant System Algorithm For The Travelling Salesman Problem

Algorithm from the Book **Ant Colony Optimization by Marco Dorigo and Thomas Stützle**

In [89]:
#----------------------------------------
# Chia E. Tungom
# chemago99@yahoo.com
# July 30th 2020
# Shenzhen China
#----------------------------------------

In [90]:
# import libraries

import numpy as np
import math
import random

# Roulette Wheel for Probability

In [91]:
# probability and selection using roulette wheel 

# Calculate cumulative sum for roulette wheel 
def PPi(pi):
    n = len(pi)
    ans = []
    
    for i in range(n):
        ans.append( sum([i for i in pi[:i+1]]) )
    return ans 

# Roulette Wheel 
def Roulette(ppi, Pop):
    n = len(ppi)
    ra = random.random()
    
    for i in range(n):
        if i == 0 :
            if ra > 0 and ra < ppi[i]:
                return Pop[i]
        else:
            if ra > ppi[i-1] and ra < ppi[i]:
                return Pop[i]

# Calculate Route Cost

In [92]:
# calculate cost of each route
def Cost(Pop, Cmatrix):
    Cvector = []
    
    for i in range(len(Pop)):
        route = Pop[i]
        cost = 0
        
        for j in range(len(route)-1):
            cost += Cmatrix[route[j]][route[j+1]]
        cost += Cmatrix[route[0]][route[len(route)-1]]   
        Cvector.append(cost)
        
    return Cvector

# Sort Chromosomes from Best to Worst
def Rank(Pop, graph):
    """ returns sorted Ants """
    cost = Cost(Pop, graph)
    ans = [ x for _,x in sorted(zip(cost, Pop))]
    #cost = [ _ for _,x in sorted(zip(cost, Pop))]  # need to use the true cost
    return ans


# fitness function 1
def F1(sortedpop):
    bb= 0.3
    F = []   
    for i in range(1,len(sortedpop)+1):
        F.append( bb * ( (1-bb)**(i-1) ) )
        
    return F

# fitness function 1
def F2(sortedpop):
    n = len(sortedpop)
    F = []
    for i in range(1,n+1):
        F.append( (n-i+1)/n )
        
    return F

# Initialize Pheromone 

In [93]:

def TSPher(graph):
    
    n = len(graph)
    route = [i for i in range(n)]
    random.shuffle(route)
    
    cost = Cost([route], graph)[0]
    pher = [[cost for i in range(n)] for j in range(n)]
    
    return pher

# Calculate Transition Probability

In [94]:
# Transition Probability
def TransitionP(connections, alpha, beta):
    """ Takes a list of tuples [(pheromone, distance, hob), ... , (pheromone, distance, hob)]
        return trasition probability for each hob [(probability, hob), ... , (probability, hob)]"""
    
    total = sum( [ (i[0]**alpha)*(i[1]**beta) for i in connections])    # sum heuristic and pheromone info
    prob = [( (((i**alpha)*(j**beta))/total) , k) for i,j,k in connections ]  # probability of each hob
    
    return prob

# pheromone, distance and hob information
def DPH(phermatrix, graph, connections, preshob):
    """ DPH(Distance Pheromone hob)
        takes a pheromone matrix
        graph matrix 
        connections in graph to use 
        present hob in graph
        and return the pheromone, distance and hob [(pheromone, distance, hob), ... , (pheromone, distance, hob)]"""
    
    dph = []
    for i in range(len(connections)):
        if connections[i] > 0:
            dph.append( (phermatrix[preshob][i], 1/graph[preshob][i], i) )
            
    #dph = [(phermatrix[preshob][i], graph[preshob][i], i) for i in range(len(connections)) if connections[i] > 0 ]
    
    return dph

def Disconnect(visited, connections):
    """ Disconnects already visited hobs
    visited: already visited hobs
    connections: connection vector with 0: no connection x: length of connection"""
    
    for i in visited:
        connections[i] = 0
        #print(i, visited, connections)
    #print("DONE++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++DONE")
    return connections

# Find Path 

In [95]:
def FindPath(phermatrix, graph, takeoff, alpha = 2, beta = 2):
    
    path = []
    path.append(takeoff)
    
    while len(path) != len(graph):
        
        Cons = graph[takeoff].copy()
        tabu = path.copy()
        Cons = Disconnect(tabu, Cons)
        HP = DPH(phermatrix, graph, Cons, takeoff)
        prob = TransitionP(HP, alpha, beta)
        takeoff = Roulette(PPi([i[0] for i in prob]), [i[1] for i in prob])
        
        path.append(takeoff)
 
    return path

# Update Pheromone 

In [181]:
# Update Pheromone in visited arcs

def UpPheromone(route, matrix):
    """route : route taken
    matrix: pheromone matrix"""
    
    tau = 1/Cost([route], matrix)[0]
    #tau = 1
    
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += tau
        matrix[route[i+1]][route[i]] += tau
    
    # add pheromone to last and first route for TSP
    matrix[route[-1]][route[0]] += tau
    matrix[route[0]][route[-1]] += tau

    return matrix


# Ant Cycle
def UpPheromoneANTC(k, route, matrix, Q = 1):
    """route : route taken
    matrix: pheromone matrix
    Q: a constant """
    
    n = len(route)
    #tau = Q/Cost([route], matrix)[0]
    
    tau = (n-k+1)/n
    #tau = 1
    
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += tau
        matrix[route[i+1]][route[i]] += tau
    
    # add pheromone to last and first route for TSP
    matrix[route[-1]][route[0]] += tau
    matrix[route[0]][route[-1]] += tau

    return matrix

# Ant Density
def UpPheromoneANTD(route, matrix, Q = 1):
    """route : route taken
    matrix: pheromone matrix
    Q: a constant """
   
    for i in range(len(route)-1):
        
        matrix[route[i]][route[i+1]] += Q
        matrix[route[i+1]][route[i]] += Q
    
    # add pheromone to last and first route for TSP
    matrix[route[-1]][route[0]] += Q
    matrix[route[0]][route[-1]] += Q

    return matrix

# Ant Quantity
def UpPheromoneANTQ(route, matrix, Q = 1):
    """route : route taken
    matrix: pheromone matrix
    Q: a constant """
   
    for i in range(len(route)-1):
        
        tau = Q/matrix[route[i]][route[i+1]]
        matrix[route[i]][route[i+1]] += tau
        matrix[route[i+1]][route[i]] += tau
    
    # add pheromone to last and first route for TSP
    tau = Q/matrix[route[-1]][route[0]]
    matrix[route[-1]][route[0]] += tau
    matrix[route[0]][route[-1]] += tau

    return matrix

# Evaporate Pheromone 

In [182]:
def EvPheromone(matrix, rho):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j] > 0:
                matrix[i][j] = (1-rho)*matrix[i][j]
            else:
                matrix[i][j] = math.exp(-5)
    return matrix

def EvPheromoneNeg(matrix, rho):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            matrix[i][j] = (1-rho)*matrix[i][j]
        
    return matrix

# Generate a Toy Symmetric TSP Problem 

In [183]:
# A random graph
def Graph(dim, bounds):
    b = [(i, str(i)) for i in range(dim)]   
    matrix = []
    
    for i,j in b:
        j=[]
        if i == 0:
            j.append(0)
            matrix.append(j)
        else:
            for k in range(i+1):
                if k == i:
                    j.append(0)
                else:
                    j.append(random.choice([random.randint(bounds[0], bounds[1])]))  # route length
            matrix.append(j)
            
    M = matrix        
    for i in range(len(M)):
        
        for j in range(1,len(M)-len(M[i])+1):
            M[i].append(M[i+j][i]) 
            
    return M

# Ant System Algorithm

In [188]:
def AS(Ngraph, ants, alpha = 1, beta = 3, rho = 0.5, runs = 50, Pher = None):
    
    if Pher == None:
        Pher = TSPher(Ngraph) 
        
    starthob = random.choice([i for i in range(len(Ngraph))])
    i = 0
    while i < runs:
        Ants = []
        #starthob = random.choice([i for i in range(len(Ngraph))])
        
        for j in range(ants):
            stof = FindPath(Pher, Ngraph, starthob, alpha, beta)
            Ants.append(stof)
            
        Ranked = Rank(Ants, Ngraph)
        
        #Pheromone update
     
        for k, j in enumerate(Ants):
            #Pher = UpPheromone(j, Pher)
            UpPheromoneANTC(k, j, Pher, 1)  # ant cycle
            #UpPheromoneANTD(j, Pher, 1)  # ant density
            #UpPheromoneANTQ(j, Pher, 1)  # Ant Quantity
            
        #Pheromone update  
        #EvPheromone(Pher, rho)
        EvPheromoneNeg(Pher, rho)
        
        if i%int(runs*0.05) == 0:
            C = Cost(Ants, Ngraph)
            print("In Generation ", i, " The Minimum Cost Found is = ", min(C), "  and the Maximum Cost Found is = ", max(C))
            
        i += 1 

    C = Cost(Ants, Ngraph)    
    print("Minimum Cost Found = ", min(C), "Maximum Cost Found = ", max(C))
    return np.array(Ants), min(C) #, np.array(Pher)

# Example on the Toy Problem

In [189]:
# Toy Network graph 
D = 10
bounds = (2,10)
random.seed(10)
graph = Graph(D, bounds)

np.array(graph)

array([[ 0,  2,  9,  9,  7,  8,  5, 10,  6,  9],
       [ 2,  0,  5,  4,  5,  6,  8,  8, 10,  6],
       [ 9,  5,  0, 10,  2,  9,  2,  3,  4, 10],
       [ 9,  4, 10,  0,  4,  6,  4, 10,  7,  3],
       [ 7,  5,  2,  4,  0,  4,  6,  5,  3,  9],
       [ 8,  6,  9,  6,  4,  0,  5,  5,  9,  4],
       [ 5,  8,  2,  4,  6,  5,  0,  2,  5,  4],
       [10,  8,  3, 10,  5,  5,  2,  0,  8,  7],
       [ 6, 10,  4,  7,  3,  9,  5,  8,  0,  8],
       [ 9,  6, 10,  3,  9,  4,  4,  7,  8,  0]])

In [190]:
# Find Shortest Path with the Algorithm

ants = len(graph)
alpha = 1
beta = 3
rho = 0.1
runs = 200

AS(graph, ants, alpha, beta, rho, runs)

In Generation  0  The Minimum Cost Found is =  34   and the Maximum Cost Found is =  47
In Generation  10  The Minimum Cost Found is =  36   and the Maximum Cost Found is =  47
In Generation  20  The Minimum Cost Found is =  34   and the Maximum Cost Found is =  46
In Generation  30  The Minimum Cost Found is =  33   and the Maximum Cost Found is =  40
In Generation  40  The Minimum Cost Found is =  34   and the Maximum Cost Found is =  45
In Generation  50  The Minimum Cost Found is =  33   and the Maximum Cost Found is =  38
In Generation  60  The Minimum Cost Found is =  33   and the Maximum Cost Found is =  34
In Generation  70  The Minimum Cost Found is =  33   and the Maximum Cost Found is =  37
In Generation  80  The Minimum Cost Found is =  33   and the Maximum Cost Found is =  34
In Generation  90  The Minimum Cost Found is =  33   and the Maximum Cost Found is =  46
In Generation  100  The Minimum Cost Found is =  33   and the Maximum Cost Found is =  38
In Generation  110  T

(array([[6, 7, 2, 4, 8, 0, 1, 3, 9, 5],
        [6, 7, 2, 4, 8, 0, 1, 3, 9, 5],
        [6, 7, 2, 4, 8, 0, 1, 3, 9, 5],
        [6, 7, 2, 4, 8, 0, 1, 3, 9, 5],
        [6, 7, 2, 4, 8, 0, 1, 3, 9, 5],
        [6, 7, 2, 4, 8, 0, 1, 3, 9, 5],
        [6, 5, 7, 2, 4, 8, 0, 1, 3, 9],
        [6, 7, 2, 4, 8, 0, 1, 3, 9, 5],
        [6, 7, 2, 4, 8, 0, 1, 3, 9, 5],
        [6, 7, 2, 4, 8, 0, 1, 3, 9, 5]]), 34)

# Soving Real TSP problems 

In [ ]:
# att48

import RouteMatrix as RM
path = './Data/att48.txt'
graph = RM.TSRM(path)


ants = len(graph)
alpha = 1
beta = 3
rho = 0.1
runs = 200

AS(graph, ants, alpha, beta, rho, runs)

In Generation  0  The Minimum Cost Found is =  48230.83432349601   and the Maximum Cost Found is =  69614.4385166183
In Generation  10  The Minimum Cost Found is =  44399.881106471   and the Maximum Cost Found is =  69410.32666101538
In Generation  20  The Minimum Cost Found is =  46736.89880696411   and the Maximum Cost Found is =  71961.71856220896
In Generation  30  The Minimum Cost Found is =  47793.15731871302   and the Maximum Cost Found is =  66058.87753378235
In Generation  40  The Minimum Cost Found is =  46040.71610856766   and the Maximum Cost Found is =  66100.16261297622
In Generation  50  The Minimum Cost Found is =  46221.091379700796   and the Maximum Cost Found is =  67814.3197457046
In Generation  60  The Minimum Cost Found is =  46897.433414974876   and the Maximum Cost Found is =  66671.17954183005


In [ ]:
import RouteMatrix as RM
path = './Data/burma14.txt'
graph = RM.TSRM(path)


ants = len(graph)
#ants = 100
alpha = 1
beta = 3
rho = 0.1
runs = 100

AS(graph, ants, alpha, beta, rho, runs)